In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder


In [2]:
df=pd.read_csv(r"/content/energy-pop-exposure-nuclear-plants-locations_plants.csv")

In [3]:
df.dropna(axis='columns', inplace=True)

In [4]:
df

,FID,Country,Plant,NumReactor,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,...,p10r_600,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300
0,0,SWEDEN,AGESTA,1,59.206022,18.082872,187382000,188684000,188250000,121178000.0,...,8972550.0,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190
1,1,SPAIN,ALMARAZ,2,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,...,19453700.0,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340
2,2,BRAZIL,ANGRA,3,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,...,18605600.0,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300
3,3,UNITED STATES OF AMERICA,ARKANSAS ONE,2,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,...,9498240.0,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070
4,4,SPAIN,ASCO,2,41.200000,0.566670,271854000,287134000,308922000,190825000.0,...,17594700.0,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,CHINA,YANGJIANG,3,21.708333,112.261111,557032000,629419000,679747000,250848000.0,...,86375000.0,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500
272,272,UNITED STATES OF AMERICA,YANKEE,1,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,...,9951640.0,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260
273,273,"KOREA, REPUBLIC OF",YONGGWANG,6,35.411130,126.416190,649425000,710033000,750775000,404604000.0,...,35715500.0,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200
274,274,UKRAINE,ZAPOROZHE,6,47.511809,34.585460,273432000,274834000,272561000,162035000.0,...,22931000.0,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510


In [5]:
columns_to_drop = ['FID', 'Plant', 'NumReactor']
df.drop(columns=columns_to_drop, inplace=True)
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10r_600,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,8972550.0,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,19453700.0,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,18605600.0,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,9498240.0,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,17594700.0,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,86375000.0,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,9951640.0,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,35715500.0,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,22931000.0,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510


In [6]:
# Label Encoding
label_encoder = LabelEncoder()  # Create a LabelEncoder instance
categorical_cols = [ 'Country']  # Specify the categorical columns to be encoded

for col in categorical_cols:
    df[col + '_encoded'] = label_encoder.fit_transform(df[col])  # Apply label encoding to each column

print("Label Encoded Data:")
df

Label Encoded Data:


,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          276 non-null    object 
 1   Latitude         276 non-null    float64
 2   Longitude        276 non-null    float64
 3   p90_1200         276 non-null    int64  
 4   p00_1200         276 non-null    int64  
 5   p10_1200         276 non-null    int64  
 6   p90u_1200        276 non-null    float64
 7   p00u_1200        276 non-null    float64
 8   p10u_1200        276 non-null    float64
 9   p90r_1200        276 non-null    int64  
 10  p00r_1200        276 non-null    int64  
 11  p10r_1200        276 non-null    int64  
 12  p90_30           276 non-null    float64
 13  p00_30           276 non-null    float64
 14  p10_30           276 non-null    float64
 15  p90u_30          276 non-null    float64
 16  p00u_30          276 non-null    float64
 17  p10u_30         

In [8]:
X=df[["p90_1200","p00_1200","p10_1200"]]
X

,p90_1200,p00_1200,p10_1200
0,187382000,188684000,188250000
1,136675000,147718000,163429000
2,99195200,113894000,127898000
3,117830000,132729000,146482000
4,271854000,287134000,308922000
...,...,...,...
271,557032000,629419000,679747000
272,123430000,131527000,145162000
273,649425000,710033000,750775000
274,273432000,274834000,272561000


In [9]:
import numpy as np
import pandas as pd

def estimate_coef(x, y):
    # number of observations/points
    n = np.size(x)

    # mean of x and y vector
    m_x = np.mean(x)
    m_y = np.mean(y)

    # calculating cross-deviation and deviation about x
    SS_xy = np.sum(y*x) - n*m_y*m_x
    SS_xx = np.sum(x*x) - n*m_x*m_x

    # calculating regression coefficients
    b_1 = SS_xy / SS_xx
    b_0 = m_y - b_1*m_x

    return (b_0, b_1)

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

M = []
N = []

for i in range(0, 276):
    x = np.array([0.1, 0.15, 0.20])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])
    coef = estimate_coef(x, y)
        # Accessing the regression coefficients
    b_0, b_1 = coef
    print(f"Regression Coefficients for row {i}: b_0 = {b_0}, b_1 = {b_1}")
    M.append(b_0)
    N.append(b_1)

Regression Coefficients for row 0: b_0 = 186803333.33333334, b_1 = 8679999.999999968
Regression Coefficients for row 1: b_0 = 109143000.00000015, b_1 = 267539999.99999902
Regression Coefficients for row 2: b_0 = 70608200.00000016, b_1 = 287027999.9999989
Regression Coefficients for row 3: b_0 = 89369000.00000016, b_1 = 286519999.9999989
Regression Coefficients for row 4: b_0 = 233701333.33333352, b_1 = 370679999.9999986
Regression Coefficients for row 5: b_0 = 44300533.33333342, b_1 = 161239999.9999994
Regression Coefficients for row 6: b_0 = 354666833.33333343, b_1 = 226729999.99999917
Regression Coefficients for row 7: b_0 = 146591666.66666663, b_1 = -80039999.9999997
Regression Coefficients for row 8: b_0 = 334174000.00000006, b_1 = 126459999.99999954
Regression Coefficients for row 9: b_0 = 135709000.00000018, b_1 = 332979999.9999988
Regression Coefficients for row 10: b_0 = 285644666.66666675, b_1 = 79879999.9999997
Regression Coefficients for row 11: b_0 = 329247833.3333335, b_1 

In [10]:
p20_1200=[]
for j in range(0,276):
    y=M[j]+N[j]*0.25
    p20_1200.append(y)

In [11]:
p20_1200=np.fix(p20_1200)
p20_1200

array([1.88973333e+08, 1.76027999e+08, 1.42365199e+08, 1.60998999e+08,
       3.26371333e+08, 8.46105330e+07, 4.11349333e+08, 1.26581666e+08,
       3.65788999e+08, 2.18953999e+08, 3.05614666e+08, 3.96635333e+08,
       5.38374000e+07, 3.05137999e+08, 4.35906666e+08, 4.33829999e+08,
       1.98190333e+08, 2.76039000e+05, 3.76270333e+08, 1.54162666e+08,
       4.15294000e+08, 7.35279330e+07, 3.80831666e+08, 3.45151999e+08,
       2.03938999e+08, 3.11338999e+08, 3.96490999e+08, 2.06890333e+08,
       2.04790333e+08, 3.94534666e+08, 1.93838666e+08, 4.41982666e+08,
       1.55628866e+08, 1.76452666e+08, 1.81855666e+08, 2.04021333e+08,
       4.53056999e+08, 2.23660333e+08, 4.18998666e+08, 8.97854333e+08,
       2.94826333e+08, 6.20960333e+08, 2.43572666e+08, 7.68232333e+08,
       2.71011333e+08, 7.03092666e+08, 3.64281333e+08, 4.00614333e+08,
       3.87241999e+08, 2.01744999e+08, 2.32434666e+08, 4.70343660e+07,
       1.02384733e+08, 1.36467999e+08, 4.43616666e+08, 4.39727333e+08,
      

In [12]:
df['p20_1200LR'] = p20_1200.tolist()
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0


In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

A = []
B = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    # Create a linear regression model
    model = LinearRegression()
    model.fit(x, y)

    # Access the regression coefficients
    b_0 = model.intercept_
    b_1 = model.coef_[0]

    print(f"Regression Coefficients for row {i}: b_0 = {b_0}, b_1 = {b_1}")
    A.append(b_0)
    B.append(b_1)

Regression Coefficients for row 0: b_0 = 186803333.33333334, b_1 = 8680000.0
Regression Coefficients for row 1: b_0 = 109142999.99999999, b_1 = 267540000.00000012
Regression Coefficients for row 2: b_0 = 70608200.0, b_1 = 287028000.00000006
Regression Coefficients for row 3: b_0 = 89369000.0, b_1 = 286520000.00000006
Regression Coefficients for row 4: b_0 = 233701333.3333333, b_1 = 370680000.00000006
Regression Coefficients for row 5: b_0 = 44300533.33333332, b_1 = 161240000.00000006
Regression Coefficients for row 6: b_0 = 354666833.3333333, b_1 = 226730000.00000006
Regression Coefficients for row 7: b_0 = 146591666.66666666, b_1 = -80040000.00000001
Regression Coefficients for row 8: b_0 = 334174000.0, b_1 = 126460000.00000001
Regression Coefficients for row 9: b_0 = 135709000.0, b_1 = 332980000.0
Regression Coefficients for row 10: b_0 = 285644666.6666667, b_1 = 79880000.00000001
Regression Coefficients for row 11: b_0 = 329247833.3333333, b_1 = 269550000.00000006
Regression Coeffic

In [14]:
p20_1200LIB=[]
for j in range(0,276):
    y=A[j]+B[j]*0.25
    p20_1200LIB.append(y)
p20_1200LIB=np.fix(p20_1200LIB)
p20_1200LIB
#USING LIBRARY IS LIB
df['p20_1200LIB'] = p20_1200LIB.tolist()
df


,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0


In [15]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

D = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(x, y)
    y_pred = regressor.predict([[0.25]])
    D.append(y_pred)

In [16]:
D=np.fix(D)
df['P20_1200DT'] = D.tolist()
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,[188250000.0]
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,[163429000.0]
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,[127898000.0]
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,[146482000.0]
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,[308922000.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,[679747000.0]
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,[145162000.0]
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,[750775000.0]
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,[272561000.0]


In [17]:
df.loc[:, 'P20_1200DT'] = D.tolist()


<ipython-input-17-df8a58fe72f3>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'P20_1200DT'] = D.tolist()


In [18]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0


In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

s = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    # Create a Ridge regression model
    regressor = Ridge(alpha=1.0)
    regressor.fit(x, y)

    x_test = np.array([[0.25]])
    y_pred = regressor.predict(x_test)
    s.append(y_pred)


In [20]:
df['P20_1200ridge'] = s
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,[188109651.74129355]
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,[149407104.47761193]
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,[113805200.0]
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,[132489547.26368159]
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,[289487751.2437811]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,[622676522.3880597]
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,[133481119.40298507]
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,[703915228.8557214]
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,[273604666.6666667]


In [21]:
df.loc[:, 'P20_1200ridge'] = s


<ipython-input-21-ae2148f48e4c>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'P20_1200ridge'] = s


In [22]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08


In [23]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor



# Create a new column 'Predicted_p20_1200' and set it to 0
df['Predicted_p20_1200'] = 0

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values

# Target variable
y = df['p20_1200LR'].values

# Create a KNN regressor object
knn = KNeighborsRegressor(n_neighbors=5)

# Fit the model to the data
knn.fit(X, y)

# Make predictions for the input features
predictions = knn.predict(X)

# Assign the predicted values to the 'Predicted_p20_1200knn' column
df['Predicted_p20_1200knn'] = predictions


In [24]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200,Predicted_p20_1200knn
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,0,184029199.6
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,0,176228332.6
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,0,135754986.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,0,157149732.6
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,0,314587132.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,0,730542466.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,0,154687199.4
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,0,821921466.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,0,281498999.8


In [25]:
# Drop the 'knn2' and 'Predicted_p20_1200' columns
df = df.drop([ 'Predicted_p20_1200'], axis=1)
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8


In [26]:
import pandas as pd
from sklearn.neural_network import MLPRegressor

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values

# Target variable
y = df['p20_1200LR'].values

# Create an ANN regressor object
regressor = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)

# Fit the model to the data
regressor.fit(X, y)

# Make predictions for the input features
predictions = regressor.predict(X)

# Add the predictions as a new column in the DataFrame
df['ANN2'] = predictions


In [27]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08


In [28]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values
y = df['p20_1200LR'].values

# Perform feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create an ElasticNet regressor object
regressor = ElasticNet(alpha=0.5, l1_ratio=0.5)

# Fit the model to the scaled data
regressor.fit(X_scaled, y)

# Make predictions for the scaled data
predictions = regressor.predict(X_scaled)

# Add the predictions as a new column in the DataFrame
df['en'] = predictions


In [29]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08,2.132813e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08,1.716552e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08,1.327434e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08,1.532078e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08,3.253859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08,6.930402e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08,1.540555e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08,7.813568e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08,3.068750e+08


In [30]:
df = df.drop('Country', axis=1)
df

,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,p00r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,66384500,...,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08,2.132813e+08
1,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,53774400,...,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08,1.716552e+08
2,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,36072700,...,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08,1.327434e+08
3,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,27947300,...,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08,1.532078e+08
4,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,86668900,...,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08,3.253859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,341528000,...,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08,6.930402e+08
272,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,21426000,...,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08,1.540555e+08
273,35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,269049000,...,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08,7.813568e+08
274,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,111341000,...,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08,3.068750e+08


In [31]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming 'df' is your dataset

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize each column in the dataset
for column in df.columns:
    # Reshape the data to 2D array before scaling
    column_data = df[column].values.reshape(-1, 1)
    df[column] = scaler.fit_transform(column_data)
df

,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,p00r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,0.913304,0.489398,0.226290,0.187065,0.158115,0.249432,0.223774,0.210121,0.109688,0.090250,...,0.015467,0.013903,0.848485,0.137258,0.137258,0.158115,0.186554,0.134683,0.190114,0.184691
1,0.723163,0.407610,0.164938,0.146380,0.137234,0.180179,0.171884,0.178023,0.081266,0.073026,...,0.065309,0.062053,0.818182,0.127841,0.127841,0.137234,0.148104,0.127897,0.150417,0.147317
2,0.107433,0.274295,0.119590,0.112788,0.107343,0.139440,0.142382,0.150086,0.051805,0.048847,...,0.073693,0.070927,0.090909,0.103355,0.103355,0.107343,0.112735,0.092690,0.113736,0.112380
3,0.679075,0.106545,0.142137,0.131494,0.122977,0.190616,0.191717,0.198614,0.041441,0.037749,...,0.019304,0.018286,1.000000,0.116909,0.116909,0.122977,0.131297,0.111301,0.132749,0.130754
4,0.736806,0.429153,0.328496,0.284839,0.259634,0.392819,0.366816,0.369266,0.134377,0.117956,...,0.031856,0.030315,0.818182,0.237201,0.237201,0.259634,0.287269,0.248253,0.292289,0.285344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,0.545747,0.813314,0.673545,0.624773,0.571601,0.516392,0.526803,0.531251,0.509313,0.466063,...,0.326416,0.299600,0.181818,0.541551,0.541551,0.571601,0.618279,0.610088,0.624560,0.615443
272,0.751782,0.176372,0.148912,0.130300,0.121866,0.212784,0.201452,0.208687,0.032836,0.028842,...,0.035898,0.033932,1.000000,0.112622,0.112622,0.121866,0.132282,0.109159,0.134545,0.131515
273,0.680063,0.861999,0.785335,0.704834,0.631356,0.832940,0.806956,0.798312,0.407129,0.367065,...,0.064481,0.057768,0.515152,0.585180,0.585180,0.631356,0.698986,0.689577,0.708223,0.694738
274,0.798676,0.546157,0.330406,0.272623,0.229045,0.333547,0.299158,0.278708,0.184946,0.151655,...,0.061535,0.048980,0.939394,0.198188,0.198188,0.229045,0.271489,0.219471,0.276624,0.268724


In [32]:
from sklearn.decomposition import PCA

# Separate features and target variables
target_columns = ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']
features = df.drop(target_columns, axis=1)
target_variables = df[target_columns]

# Apply dimensionality reduction using PCA
pca = PCA(n_components=5)  # Set the desired number of components
features_reduced = pca.fit_transform(features)

# Create a new DataFrame for the reduced features
df_reduced = pd.DataFrame(features_reduced, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

# Concatenate the reduced features with the target variables
df_combined = pd.concat([df_reduced, target_variables], axis=1)


In [33]:
df_combined

,PC1,PC2,PC3,PC4,PC5,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,-0.802412,0.070108,0.533996,-0.213804,-0.027186,0.137258,0.137258,0.158115,0.186554,0.134683,0.190114,0.184691
1,-0.809580,0.159435,-0.019536,0.127898,0.036270,0.127841,0.127841,0.137234,0.148104,0.127897,0.150417,0.147317
2,-0.034519,-0.280523,-0.363478,0.023808,0.644185,0.103355,0.103355,0.107343,0.112735,0.092690,0.113736,0.112380
3,-1.078497,0.039058,0.073401,0.177846,-0.135125,0.116909,0.116909,0.122977,0.131297,0.111301,0.132749,0.130754
4,-0.501050,0.171726,0.019281,0.036630,-0.029960,0.237201,0.237201,0.259634,0.287269,0.248253,0.292289,0.285344
...,...,...,...,...,...,...,...,...,...,...,...,...
271,1.232798,0.643078,-0.294524,0.178935,0.096370,0.541551,0.541551,0.571601,0.618279,0.610088,0.624560,0.615443
272,-0.369139,-0.368610,-0.332430,0.160723,-0.245654,0.112622,0.112622,0.121866,0.132282,0.109159,0.134545,0.131515
273,0.767564,0.617158,-0.073923,-0.267815,-0.203559,0.585180,0.585180,0.631356,0.698986,0.689577,0.708223,0.694738
274,-0.478618,0.140030,0.142593,0.061935,-0.085718,0.198188,0.198188,0.229045,0.271489,0.219471,0.276624,0.268724


In [34]:
# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)
# Assuming you have multiple target variables named 'target_variable1', 'target_variable2', etc.
target_variables =  ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']

In [35]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
rmse_list = []
mae_list = []
r2_list = []
explained_variance_list = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=0.2,
                                                    random_state=42)

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the linear regression model
    model = LinearRegression()

    # Fit the model on the training fold
    model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        rmse_list.append(rmse_val)
        mae_list.append(mae_val)
        r2_list.append(r2_val)
        explained_variance_list.append(explained_variance_val)

        print("RMSE for", target_variable, ":", rmse_val)
        print("MAE for", target_variable, ":", mae_val)
        print("R2 score for", target_variable, ":", r2_val)
        print("Explained variance score for", target_variable, ":", explained_variance_val)


RMSE for p20_1200LR : 0.02793994345793301
MAE for p20_1200LR : 0.02096443414882185
R2 score for p20_1200LR : 0.9268876192725677
Explained variance score for p20_1200LR : 0.928535672789794
RMSE for p20_1200LIB : 0.02793994339393536
MAE for p20_1200LIB : 0.020964434040737376
R2 score for p20_1200LIB : 0.9268876193501789
Explained variance score for p20_1200LIB : 0.9285356728008662
RMSE for P20_1200DT : 0.02692759581629581
MAE for P20_1200DT : 0.020536683694202452
R2 score for P20_1200DT : 0.942313744154733
Explained variance score for P20_1200DT : 0.9435791809094096
RMSE for P20_1200ridge : 0.030085227031943513
MAE for P20_1200ridge : 0.023581887458989648
R2 score for P20_1200ridge : 0.9425737262379464
Explained variance score for P20_1200ridge : 0.9435437978233178
RMSE for Predicted_p20_1200knn : 0.030839402511352768
MAE for Predicted_p20_1200knn : 0.022778802937526047
R2 score for Predicted_p20_1200knn : 0.9336168674358901
Explained variance score for Predicted_p20_1200knn : 0.93674451

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
rmse_list = []
mae_list = []
r2_list = []
explained_variance_list = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=0.2,
                                                    random_state=42)

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the linear regression model
    model = LinearRegression()

    # Fit the model on the training fold
    model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        rmse_list.append(rmse_val)
        mae_list.append(mae_val)
        r2_list.append(r2_val)
        explained_variance_list.append(explained_variance_val)

        print("RMSE for", target_variable, ":", rmse_val)
        print("MAE for", target_variable, ":", mae_val)
        print("R2 score for", target_variable, ":", r2_val)
        print("Explained variance score for", target_variable, ":", explained_variance_val)

# Define the parameter grid for hyperparameter tuning
param_grid = {}

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=n_splits)

# Fit the model on the training data
grid_search.fit(X_train, y_train)

# Get the best model after hyperparameter tuning
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_test = best_model.predict(X_test)

# Calculate the evaluation metrics for each target variable on the test set
for i, target_variable in enumerate(target_variables):
    rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test[:, i]))
    mae_test = mean_absolute_error(y_test[target_variable], y_pred_test[:, i])
    r2_test = r2_score(y_test[target_variable], y_pred_test[:, i])
    explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test[:, i])

    print("Best Model RMSE for", target_variable, ":", rmse_test)
    print("Best Model MAE for", target_variable, ":", mae_test)
    print("Best Model R2 score for", target_variable, ":", r2_test)
    print("Best Model Explained variance score for", target_variable, ":", explained_variance_test)

# Get the best hyperparameters
best_hyperparameters = grid_search.best_params_
print("Best Hyperparameters:", best_hyperparameters)


RMSE for p20_1200LR : 0.030325637310497967
MAE for p20_1200LR : 0.023114538554719843
R2 score for p20_1200LR : 0.9582252686804474
Explained variance score for p20_1200LR : 0.9609740756362805
RMSE for p20_1200LIB : 0.030325637294607737
MAE for p20_1200LIB : 0.02311453849595325
R2 score for p20_1200LIB : 0.9582252686431822
Explained variance score for p20_1200LIB : 0.9609740755654421
RMSE for P20_1200DT : 0.03142108782933268
MAE for P20_1200DT : 0.02414186467804703
R2 score for P20_1200DT : 0.9602977604949311
Explained variance score for P20_1200DT : 0.9622208785208174
RMSE for P20_1200ridge : 0.036127683265974785
MAE for P20_1200ridge : 0.028288635334645266
R2 score for P20_1200ridge : 0.9563317607931411
Explained variance score for P20_1200ridge : 0.9573276172454921
RMSE for Predicted_p20_1200knn : 0.03983488583376252
MAE for Predicted_p20_1200knn : 0.031068389689261802
R2 score for Predicted_p20_1200knn : 0.9471237381005037
Explained variance score for Predicted_p20_1200knn : 0.949553

In [39]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
rmse_list = []
mae_list = []
r2_list = []
explained_variance_list = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=0.2,
                                                    random_state=42)

# Define the hyperparameters grid for tuning
param_grid = {
    'fit_intercept': [True, False],  # Whether to calculate the intercept
    'copy_X': [True, False],  # Whether to copy the input data
}

# Initialize the linear regression model
model = LinearRegression()

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=n_splits, scoring='neg_mean_squared_error', refit=True)

# Fit the model on the training data
grid_search.fit(X_train, y_train)

# Get the best model after hyperparameter tuning
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_test = best_model.predict(X_test)

# Calculate the evaluation metrics for each target variable on the test set
evaluation_metrics = []
for i, target_variable in enumerate(target_variables):
    rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test[:, i]))
    mae_test = mean_absolute_error(y_test[target_variable], y_pred_test[:, i])
    r2_test = r2_score(y_test[target_variable], y_pred_test[:, i])
    explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test[:, i])

    metrics = {
        'Target Variable': target_variable,
        'RMSE': rmse_test,
        'MAE': mae_test,
        'R2 Score': r2_test,
        'Explained Variance': explained_variance_test
    }
    evaluation_metrics.append(metrics)

# Convert the evaluation metrics to a pandas DataFrame
metrics_df = pd.DataFrame(evaluation_metrics)

# Print the evaluation metrics in tabular format
print(metrics_df)
print()

# Get the best hyperparameters
best_hyperparameters = grid_search.best_params_
print("Best Hyperparameters:", best_hyperparameters)


         Target Variable      RMSE       MAE  R2 Score  Explained Variance
0             p20_1200LR  0.055465  0.036339  0.908196            0.909899
1            p20_1200LIB  0.055465  0.036339  0.908196            0.909899
2             P20_1200DT  0.052610  0.034593  0.921738            0.923034
3          P20_1200ridge  0.051774  0.034255  0.931436            0.932372
4  Predicted_p20_1200knn  0.067964  0.044312  0.894583            0.897025
5                   ANN2  0.051682  0.034400  0.932541            0.933418
6                     en  0.051827  0.034209  0.930873            0.931834

Best Hyperparameters: {'copy_X': True, 'fit_intercept': True}


In [40]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Import other models as needed

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
evaluation_metrics = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=0.2,
                                                    random_state=42)

# Define the models and their respective parameter grids for tuning
models = [
    ('Linear Regression', LinearRegression(), {
        'fit_intercept': [True, False]
    }),
    ('Random Forest', RandomForestRegressor(), {
        'n_estimators': [10, 50, 100],
        'max_depth': [None, 5, 10]
    }),
    ('Support Vector Regression', SVR(), {
        'C': [1, 10, 100],
        'kernel': ['linear', 'rbf']
    })
    # Add other models and their respective parameter grids
]

# Iterate over the models
for model_name, model, param_grid in models:

    # Iterate over each target variable
    for target_variable in target_variables:

        # Perform grid search with cross-validation
        grid_search = GridSearchCV(model, param_grid, cv=n_splits, scoring='neg_mean_squared_error', refit=True)

        # Fit the model on the training data for the current target variable
        grid_search.fit(X_train, y_train[target_variable])

        # Get the best model after hyperparameter tuning
        best_model = grid_search.best_estimator_

        # Evaluate the best model on the test set for the current target variable
        y_pred_test = best_model.predict(X_test)

        # Calculate the evaluation metrics for the current target variable on the test set
        rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test))
        mae_test = mean_absolute_error(y_test[target_variable], y_pred_test)
        r2_test = r2_score(y_test[target_variable], y_pred_test)
        explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test)

        metrics = {
            'Model': model_name,
            'Target Variable': target_variable,
            'RMSE': rmse_test,
            'MAE': mae_test,
            'R2 Score': r2_test,
            'Explained Variance': explained_variance_test
        }
        evaluation_metrics.append(metrics)

# Convert the evaluation metrics to a pandas DataFrame
metrics_df = pd.DataFrame(evaluation_metrics)

# Print the evaluation metrics in tabular format
print(metrics_df)


                        Model        Target Variable      RMSE       MAE  \
0           Linear Regression             p20_1200LR  0.055465  0.036339   
1           Linear Regression            p20_1200LIB  0.055465  0.036339   
2           Linear Regression             P20_1200DT  0.052610  0.034593   
3           Linear Regression          P20_1200ridge  0.051774  0.034255   
4           Linear Regression  Predicted_p20_1200knn  0.067964  0.044312   
5           Linear Regression                   ANN2  0.051682  0.034400   
6           Linear Regression                     en  0.051827  0.034209   
7               Random Forest             p20_1200LR  0.082271  0.044370   
8               Random Forest            p20_1200LIB  0.082860  0.040866   
9               Random Forest             P20_1200DT  0.079815  0.042468   
10              Random Forest          P20_1200ridge  0.072401  0.041028   
11              Random Forest  Predicted_p20_1200knn  0.097652  0.051813   
12          

In [41]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Import other models as needed

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
evaluation_metrics = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=0.2,
                                                    random_state=42)

# Define the models and their respective parameter grids for tuning
models = [
    ('Linear Regression', LinearRegression(), {
        'fit_intercept': [True, False]
    }),
    ('Random Forest', RandomForestRegressor(), {
        'n_estimators': [10, 50, 100],
        'max_depth': [None, 5, 10]
    }),
    ('Support Vector Regression', SVR(), {
        'C': [1, 10, 100],
        'kernel': ['linear', 'rbf']
    })
    # Add other models and their respective parameter grids
]

# Iterate over the models
for model_name, model, param_grid in models:

    # Iterate over each target variable
    for target_variable in target_variables:

        # Perform grid search with cross-validation
        grid_search = GridSearchCV(model, param_grid, cv=n_splits, scoring='neg_mean_squared_error', refit=True)

        # Fit the model on the training data for the current target variable
        grid_search.fit(X_train, y_train[target_variable])

        # Get the best model after hyperparameter tuning
        best_model = grid_search.best_estimator_

        # Evaluate the best model on the test set for the current target variable
        y_pred_test = best_model.predict(X_test)

        # Calculate the evaluation metrics for the current target variable on the test set
        rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test))
        mae_test = mean_absolute_error(y_test[target_variable], y_pred_test)
        r2_test = r2_score(y_test[target_variable], y_pred_test)
        explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test)

        metrics = {
            'Model': model_name,
            'Target Variable': target_variable,
            'RMSE': rmse_test,
            'MAE': mae_test,
            'R2 Score': r2_test,
            'Explained Variance': explained_variance_test
        }
        evaluation_metrics.append(metrics)

# Convert the evaluation metrics to a pandas DataFrame
metrics_df = pd.DataFrame(evaluation_metrics)

# Calculate the mean error metrics (mean absolute error and root mean squared error)
mean_metrics_df = metrics_df.groupby('Model').mean().reset_index()
mean_metrics_df['Target Variable'] = 'Mean Error'

# Concatenate the mean error metrics with the evaluation metrics DataFrame
metrics_df = pd.concat([metrics_df, mean_metrics_df], ignore_index=True)

# Set the display options for the DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Print the evaluation metrics in tabular format
print(metrics_df.to_string(index=False))


                    Model       Target Variable     RMSE      MAE  R2 Score  Explained Variance
        Linear Regression            p20_1200LR 0.055465 0.036339  0.908196            0.909899
        Linear Regression           p20_1200LIB 0.055465 0.036339  0.908196            0.909899
        Linear Regression            P20_1200DT 0.052610 0.034593  0.921738            0.923034
        Linear Regression         P20_1200ridge 0.051774 0.034255  0.931436            0.932372
        Linear Regression Predicted_p20_1200knn 0.067964 0.044312  0.894583            0.897025
        Linear Regression                  ANN2 0.051682 0.034400  0.932541            0.933418
        Linear Regression                    en 0.051827 0.034209  0.930873            0.931834
            Random Forest            p20_1200LR 0.086294 0.044112  0.777780            0.790920
            Random Forest           p20_1200LIB 0.083422 0.043022  0.792327            0.802479
            Random Forest            P20

<ipython-input-41-0f67a7dca3ab>:75: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mean_metrics_df = metrics_df.groupby('Model').mean().reset_index()


In [42]:
# Group the DataFrame by 'Model' and calculate the mean of 'RMSE' and 'MAE'
avg_rmse = metrics_df.groupby('Model')['RMSE'].mean()
avg_mae = metrics_df.groupby('Model')['MAE'].mean()

# Print the average RMSE and MAE
print("Average RMSE:")
print(avg_rmse)

print("\nAverage MAE:")
print(avg_mae)


Average RMSE:
Model
Linear Regression            0.055255
Random Forest                0.079651
Support Vector Regression    0.057928
Name: RMSE, dtype: float64

Average MAE:
Model
Linear Regression            0.036349
Random Forest                0.043064
Support Vector Regression    0.041957
Name: MAE, dtype: float64


In [43]:
print("Target variables:", target_variables)
print("Features variables:", df_combined.columns.tolist())


Target variables: ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']
Features variables: ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']


In [51]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
results = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    df_combined[['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    test_size=0.2,
                                                    random_state=42)

# Iterate over each target variable
for i, target_variable in enumerate(['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']):
    # Define lists to store the errors for the current target variable
    target_results = []

    # Iterate over the folds
    for train_index, val_index in kf.split(X_train):
        # Get the training and validation sets
        X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val = y_train[target_variable].iloc[train_index], y_train[target_variable].iloc[val_index]

        # Initialize the SGDRegressor model
        model = SGDRegressor(max_iter=1000)

        # Fit the model on the training fold
        model.fit(X_train_fold, y_train_fold)

        # Evaluate the model on the validation set
        y_pred_val = model.predict(X_val)

        # Calculate the evaluation metrics for the target variable
        rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
        mae_val = mean_absolute_error(y_val, y_pred_val)
        r2_val = r2_score(y_val, y_pred_val)
        explained_variance_val = explained_variance_score(y_val, y_pred_val)

        # Append the results to the list for the current target variable
        target_results.append({
            'Fold': 'Validation',
            'RMSE': rmse_val,
            'MAE': mae_val,
            'R2 Score': r2_val,
            'Explained Variance': explained_variance_val
        })

    # Initialize the SGDRegressor model
    model = SGDRegressor(max_iter=1000)

    # Perform grid search with cross-validation
    param_grid = {
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
    }
    grid_search = GridSearchCV(model, param_grid, cv=n_splits, error_score='raise')

    # Fit the model on the training data
    grid_search.fit(X_train, y_train[target_variable])

    # Get the best model after hyperparameter tuning
    best_model = grid_search.best_estimator_

    # Fit the best model on the training data
    best_model.fit(X_train, y_train[target_variable])

    # Predict the test set
    y_pred_test = best_model.predict(X_test)

    # Calculate the evaluation metrics for the target variable on the test set
    rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test))
    mae_test = mean_absolute_error(y_test[target_variable], y_pred_test)
    r2_test = r2_score(y_test[target_variable], y_pred_test)
    explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test)

    # Append the results to the list for the current target variable
    target_results.append({
        'Fold': 'Test',
        'RMSE': rmse_test,
        'MAE': mae_test,
        'R2 Score': r2_test,
        'Explained Variance': explained_variance_test
    })

    # Append the results for the current target variable to the main results list
    results.extend([{
        'Target Variable': target_variable,
        **result
    } for result in target_results])

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the results in tabular form
print(results_df)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

          Target Variable        Fold      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  Validation  0.038608  0.027222  0.932926            0.933047
1              p20_1200LR  Validation  0.045973  0.028941  0.929403            0.934053
2              p20_1200LR  Validation  0.034838  0.025135  0.920130            0.921353
3              p20_1200LR  Validation  0.025848  0.019054  0.967382            0.968322
4              p20_1200LR  Validation  0.036186  0.025659  0.949619            0.951435
5              p20_1200LR        Test  0.029848  0.019247  0.973415            0.973781
6             p20_1200LIB  Validation  0.028470  0.022022  0.952226            0.953140
7             p20_1200LIB  Validation  0.055059  0.033982  0.925371            0.925384
8             p20_1200LIB  Validation  0.031944  0.022714  0.957302            0.958366
9             p20_1200LIB  Validation  0.038680  0.027649  0.926327            0.926442
10            p20_1200LIB  Valid

In [52]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
results = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    df_combined[['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    test_size=0.2,
                                                    random_state=42)

# Iterate over each target variable
for i, target_variable in enumerate(['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']):
    # Define lists to store the errors for the current target variable
    target_results = []

    # Split the training data into a training set and a validation set
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train[target_variable],
                                                                  test_size=0.2, random_state=42)

    # Initialize the SGDRegressor model
    model = SGDRegressor(max_iter=1000)

    # Perform grid search with cross-validation
    param_grid = {
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
    }
    grid_search = GridSearchCV(model, param_grid, cv=n_splits, error_score='raise')

    # Fit the model on the training data
    grid_search.fit(X_train_split, y_train_split)

    # Get the best model after hyperparameter tuning
    best_model = grid_search.best_estimator_

    # Fit the best model on the training data
    best_model.fit(X_train_split, y_train_split)

    # Predict the validation set
    y_pred_val = best_model.predict(X_val)

    # Calculate the evaluation metrics for the target variable on the validation set
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    explained_variance_val = explained_variance_score(y_val, y_pred_val)

    # Append the results to the list for the current target variable
    target_results.append({
        'Fold': 'Validation',
        'RMSE': rmse_val,
        'MAE': mae_val,
        'R2 Score': r2_val,
        'Explained Variance': explained_variance_val
    })

    # Fit the best model on the entire training data
    best_model.fit(X_train, y_train[target_variable])

    # Predict the test set
    y_pred_test = best_model.predict(X_test)

    # Calculate the evaluation metrics for the target variable on the test set
    rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test))
    mae_test = mean_absolute_error(y_test[target_variable], y_pred_test)
    r2_test = r2_score(y_test[target_variable], y_pred_test)
    explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test)

    # Append the results to the list for the current target variable
    target_results.append({
        'Fold': 'Test',
        'RMSE': rmse_test,
        'MAE': mae_test,
        'R2 Score': r2_test,
        'Explained Variance': explained_variance_test
    })

    # Append the results for the current target variable to the main results list
    results.extend([{
        'Target Variable': target_variable,
        **result
    } for result in target_results])

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the results in tabular form
print(results_df)
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the errors for each target variable
results = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    df_combined[['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    test_size=0.2,
                                                    random_state=42)

# Iterate over each target variable
for i, target_variable in enumerate(['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']):
    # Define lists to store the errors for the current target variable
    target_results = []

    # Split the training data into a training set and a validation set
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train[target_variable],
                                                                  test_size=0.2, random_state=42)

    # Initialize the SGDRegressor model
    model = SGDRegressor(max_iter=1000)

    # Perform grid search with cross-validation
    param_grid = {
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
    }
    grid_search = GridSearchCV(model, param_grid, cv=n_splits, error_score='raise')

    # Fit the model on the training data
    grid_search.fit(X_train_split, y_train_split)

    # Get the best model after hyperparameter tuning
    best_model = grid_search.best_estimator_

    # Fit the best model on the training data
    best_model.fit(X_train_split, y_train_split)

    # Predict the validation set
    y_pred_val = best_model.predict(X_val)

    # Calculate the evaluation metrics for the target variable on the validation set
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    explained_variance_val = explained_variance_score(y_val, y_pred_val)

    # Append the results to the list for the current target variable
    target_results.append({
        'Fold': 'Validation',
        'RMSE': rmse_val,
        'MAE': mae_val,
        'R2 Score': r2_val,
        'Explained Variance': explained_variance_val
    })

    # Fit the best model on the entire training data
    best_model.fit(X_train, y_train[target_variable])

    # Predict the test set
    y_pred_test = best_model.predict(X_test)

    # Calculate the evaluation metrics for the target variable on the test set
    rmse_test = np.sqrt(mean_squared_error(y_test[target_variable], y_pred_test))
    mae_test = mean_absolute_error(y_test[target_variable], y_pred_test)
    r2_test = r2_score(y_test[target_variable], y_pred_test)
    explained_variance_test = explained_variance_score(y_test[target_variable], y_pred_test)

    # Append the results to the list for the current target variable
    target_results.append({
        'Fold': 'Test',
        'RMSE': rmse_test,
        'MAE': mae_test,
        'R2 Score': r2_test,
        'Explained Variance': explained_variance_test
    })

    # Append the results for the current target variable to the main results list
    results.extend([{
        'Target Variable': target_variable,
        **result
    } for result in target_results])

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the results in tabular form
print(results_df)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

          Target Variable        Fold      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  Validation  0.014346  0.011140  0.987956            0.988343
1              p20_1200LR        Test  0.030374  0.019470  0.972469            0.972855
2             p20_1200LIB  Validation  0.014109  0.010915  0.988350            0.988713
3             p20_1200LIB        Test  0.030858  0.019670  0.971585            0.971922
4              P20_1200DT  Validation  0.012703  0.009815  0.992041            0.992243
5              P20_1200DT        Test  0.024696  0.016455  0.982755            0.982948
6           P20_1200ridge  Validation  0.015663  0.011825  0.990392            0.990464
7           P20_1200ridge        Test  0.021124  0.014536  0.988586            0.988675
8   Predicted_p20_1200knn  Validation  0.018238  0.013298  0.986308            0.986847
9   Predicted_p20_1200knn        Test  0.038344  0.025445  0.966446            0.967228
10                   ANN2  Valid

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

          Target Variable        Fold      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  Validation  0.014180  0.010955  0.988232            0.988653
1              p20_1200LR        Test  0.029931  0.019243  0.973265            0.973635
2             p20_1200LIB  Validation  0.014233  0.011053  0.988145            0.988458
3             p20_1200LIB        Test  0.030468  0.019520  0.972298            0.972614
4              P20_1200DT  Validation  0.012783  0.009788  0.991941            0.992197
5              P20_1200DT        Test  0.024988  0.016580  0.982345            0.982523
6           P20_1200ridge  Validation  0.015679  0.011839  0.990373            0.990436
7           P20_1200ridge        Test  0.021038  0.014462  0.988679            0.988766
8   Predicted_p20_1200knn  Validation  0.018269  0.013490  0.986262            0.986720
9   Predicted_p20_1200knn        Test  0.038479  0.025487  0.966210            0.967027
10                   ANN2  Valid

In [53]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Define lists to store the best parameters for each target variable
best_params = []

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    df_combined[['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']],
                                                    test_size=0.2,
                                                    random_state=42)

# Iterate over each target variable
for i, target_variable in enumerate(['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']):
    # Define lists to store the errors for the current target variable
    target_results = []

    # Split the training data into a training set and a validation set
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train[target_variable],
                                                                  test_size=0.2, random_state=42)

    # Initialize the SGDRegressor model
    model = SGDRegressor(max_iter=1000)

    # Perform grid search with cross-validation
    param_grid = {
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
    }
    grid_search = GridSearchCV(model, param_grid, cv=n_splits, error_score='raise')

    # Fit the model on the training data
    grid_search.fit(X_train_split, y_train_split)

    # Get the best model after hyperparameter tuning
    best_model = grid_search.best_estimator_

    # Append the best parameters for the current target variable
    best_params.append({
        'Target Variable': target_variable,
        'Best Parameters': grid_search.best_params_
    })

# Create a DataFrame for the best parameters
best_params_df = pd.DataFrame(best_params)

# Display the best parameters
print(best_params_df)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

         Target Variable                                    Best Parameters
0             p20_1200LR  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
1            p20_1200LIB  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
2             P20_1200DT  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
3          P20_1200ridge  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
4  Predicted_p20_1200knn  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
5                   ANN2  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
6                     en  {'alpha': 0.0001, 'learning_rate': 'adaptive',...
